# Билет 43

# Задание 2

In [1]:
import numpy as np

In [2]:
A = np.array([[8, 4, 15, 6, 9],
              [3, 16, 13, 8, 10],
              [0, 9, 18, 17, 5],
              [16, 2, 6, 0, 10],
              [18, 13, 9, 17, 8]])

In [3]:
min_x = A.min(axis=1)
min_y = A.min(axis=0)

In [4]:
B = min_x.reshape(5, -1) + min_y
B

array([[ 4,  6, 10,  4,  9],
       [ 3,  5,  9,  3,  8],
       [ 0,  2,  6,  0,  5],
       [ 0,  2,  6,  0,  5],
       [ 8, 10, 14,  8, 13]])

# Задание 3

In [5]:
import math
import numpy as np
import multiprocessing as mp

In [6]:
# пары (2, 5) и (5, 2) - считаю за одну - {2, 5}


def algorithm(ints_sample):
    limit = 0.99999999
    result = []

    n = len(ints_sample)
    seq = list(range(n))
    for i in seq:
        left = ints_sample[i]
        for j in seq[i:]:  # рассматриваем пары вне зависимости от порядка элементов
            right = ints_sample[j]
            sin_mul = math.sin(left * right)
            if sin_mul > limit:
                result.append((left, right, sin_mul))

    return result

In [7]:
%%file "src/task_3.py"
import math


def foo(chunk, ints_sample):
    limit = 0.99999999
    result = set()
    for i in chunk:
        for j in ints_sample:
            sin_mul = math.sin(i * j)
            if sin_mul > limit:
                result.add((frozenset([i, j]), sin_mul))  # оставляем уникальную пару без порядка элементов

    return result

Writing src/task_3.py


In [8]:
def blocking_mode(ints_sample):
    return algorithm(ints_sample)


def nonblocking_mode(ints_sample):
    from src.task_3 import foo

    def chunkinize(lst, n: int):
        n = len(lst) // n
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    n = mp.cpu_count()
    chunks = list(chunkinize(ints_sample, n))
    with mp.Pool(n) as pool:
        res = pool.starmap(foo, zip(chunks, [ints_sample] * n))
        return list({i for chunk in res for i in chunk})

In [9]:
ints_sample = np.arange(1, 3000 + 1)

In [10]:
blocking_result = blocking_mode(ints_sample)
print(len(blocking_result))
blocking_result[:20]

205


[(23, 2253, 0.999999999696513),
 (57, 2746, 0.9999999998635031),
 (62, 2536, 0.9999999970500172),
 (69, 751, 0.999999999696513),
 (91, 2855, 0.9999999980220771),
 (114, 1373, 0.9999999998635031),
 (115, 2253, 0.9999999924128237),
 (124, 1268, 0.9999999970500172),
 (129, 2025, 0.9999999983363994),
 (133, 2746, 0.99999999925685),
 (135, 1935, 0.9999999983363994),
 (157, 1006, 0.9999999906018032),
 (158, 2316, 0.9999999951152019),
 (193, 1896, 0.9999999951152019),
 (194, 2951, 0.9999999982003109),
 (209, 2746, 0.9999999981648745),
 (215, 1215, 0.9999999983363994),
 (219, 2149, 0.9999999901999105),
 (222, 2582, 0.9999999999999568),
 (225, 1161, 0.9999999983363994)]

In [11]:
nonblocking_result = nonblocking_mode(ints_sample)
print(len(nonblocking_result))
nonblocking_result[:20]

205


[(frozenset({1492, 1782}), 0.999999994998317),
 (frozenset({2219, 2841}), 0.9999999961980565),
 (frozenset({602, 1991}), 0.9999999999814237),
 (frozenset({753, 2007}), 0.9999999999595361),
 (frozenset({215, 1215}), 0.9999999983363994),
 (frozenset({1551, 2049}), 0.9999999984365575),
 (frozenset({409, 1910}), 0.9999999992793377),
 (frozenset({1726, 1781}), 0.9999999993232765),
 (frozenset({1336, 2769}), 0.9999999995198774),
 (frozenset({1525, 1537}), 0.9999999964929694),
 (frozenset({91, 2855}), 0.9999999980220771),
 (frozenset({443, 2704}), 0.9999999985315368),
 (frozenset({719, 1668}), 0.9999999977965826),
 (frozenset({1039, 1556}), 0.9999999939010542),
 (frozenset({288, 2710}), 0.999999995173126),
 (frozenset({1442, 2276}), 0.9999999971839069),
 (frozenset({586, 2754}), 0.9999999914573485),
 (frozenset({542, 1440}), 0.999999995173126),
 (frozenset({1598, 2968}), 0.999999996866661),
 (frozenset({129, 2025}), 0.9999999983363994)]

In [12]:
%%timeit

blocking_mode(ints_sample)

1.51 s ± 54.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit

nonblocking_mode(ints_sample)

1.26 s ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
# даже при замедлении алгоритма при параллельном решении,
# параллельное решение является более быстрым и оптимальным